In [1]:
import pymysql
conn = pymysql.connect(host='localhost',port=3306,db='lolgamequiz',user='root',passwd='root',charset='utf8')
def sele(sql):
    cur = conn.cursor()
    cur.execute(sql)
    result = list(cur.fetchall())
    result = [list(r) for r in result]
    conn.commit()
    cur.close()
    
    return result


In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [38]:
sql = 'select date,processor, dayNum, dayOrder, teamA, teamB, scoreA, scoreB, fullTeamA, fullTeamB from world2020\
 union all select date,processor, dayNum, dayOrder, teamA, teamB, scoreA, scoreB, fullTeamA, fullTeamB from world2019'
data = sele(sql)
results = pd.DataFrame(data, columns=['date', 'processor','dayNum','dayOrder', 'teamA', 'teamB', 'scoreA', 'scoreB', 'fullTeamA', 'fullTeamB'])
print(results)

sql = 'select teamName, rank, region from rank2020 where region <> "Play-In"\
 union all select teamName, rank, region from rank2019 where region <> "Play-In"'
data = sele(sql)
rank = pd.DataFrame(data, columns=['teamName', 'rank', 'region'])
print(rank)

           date   processor         dayNum dayOrder teamA teamB scoreA scoreB  \
0    2020-09-25     Play-In          Day 1        1   MAD   ITZ      1      0   
1    2020-09-25     Play-In          Day 1        2   PSG    R7      1      0   
2    2020-09-25     Play-In          Day 1        3   LGC   ITZ      1      0   
3    2020-09-25     Play-In          Day 1        4   LGD   PSG      0      1   
4    2020-09-25     Play-In          Day 1        5    TL   MAD      1      0   
..          ...         ...            ...      ...   ...   ...    ...    ...   
164  2019-10-27  Main_Event  Quarterfinals        3   SKT   SPY      3      1   
165  2019-10-27  Main_Event  Quarterfinals        4   DWG    G2      1      3   
166  2019-11-02  Main_Event     Semifinals        2    iG   FPX      1      3   
167  2019-11-03  Main_Event     Semifinals        1    G2   SKT      3      1   
168  2019-11-10  Main_Event         Finals        1    G2   FPX      0      3   

           fullTeamA       

In [39]:
# clean data
# cover data type
results = results.astype({'date':'datetime64', 'scoreA':'int64', 'scoreB':'int64', 'fullTeamA':'string', 'fullTeamB':'string'})
# fix team name
results['teamA'] = results['teamA'].apply(lambda x: x.upper())
results['teamB'] = results['teamB'].apply(lambda x: x.upper())
results['fullTeamA'] = results['fullTeamA'].apply(lambda x: x.upper())
results['fullTeamB'] = results['fullTeamB'].apply(lambda x: x.upper())
# sort
results = results.sort_values(by=['date','dayNum','dayOrder'])
results = results.reset_index()
results = results.drop(['index'], axis=1)
print(results)


####################
# rank
# region
rank = rank.astype({'rank':'int64', 'teamName':'string'})
rank['teamName'] = rank['teamName'].apply(lambda x: x.upper())
print(rank)

          date   processor         dayNum dayOrder teamA teamB  scoreA  \
0   2019-10-02     Play-In          Day 1        1    CG   UOL       0   
1   2019-10-02     Play-In          Day 1        2   DFM   SPY       0   
2   2019-10-02     Play-In          Day 1        3   MMM   UOL       1   
3   2019-10-02     Play-In          Day 1        4   DFM   ISG       0   
4   2019-10-02     Play-In          Day 1        5   MMM    CG       0   
..         ...         ...            ...      ...   ...   ...     ...   
164 2020-10-11  Main_Event          Day 8        6   DRX   TES       0   
165 2020-10-15  Main_Event  Quarterfinals        4   DWG   DRX       0   
166 2020-10-16  Main_Event  Quarterfinals        2    SN   JDG       0   
167 2020-10-17  Main_Event  Quarterfinals        1   TES   FNC       0   
168 2020-10-18  Main_Event  Quarterfinals        3   GEN    G2       0   

     scoreB           fullTeamA             fullTeamB  
0         1       CLUTCH GAMING  UNICORNS OF LOVE.CIS  

In [40]:
df = results['scoreA'].append(results['scoreB'])
df = pd.DataFrame(df)
df.columns = ['score']
gy = df['score'].groupby(df['score'])
print(gy.count())
print(results[results['scoreA']==results['scoreB']])

score
0    163
1    158
2      2
3     15
Name: score, dtype: int64
          date   processor         dayNum dayOrder teamA teamB  scoreA  \
165 2020-10-15  Main_Event  Quarterfinals        4   DWG   DRX       0   
166 2020-10-16  Main_Event  Quarterfinals        2    SN   JDG       0   
167 2020-10-17  Main_Event  Quarterfinals        1   TES   FNC       0   
168 2020-10-18  Main_Event  Quarterfinals        3   GEN    G2       0   

     scoreB      fullTeamA   fullTeamB  
165       0  DAMWON GAMING         DRX  
166       0         SUNING   JD GAMING  
167       0    TOP ESPORTS      FNATIC  
168       0          GEN.G  G2 ESPORTS  


In [41]:
# won
results['Awon'] = False
results['Bwon'] = False
results['Awon'] = results['scoreA']>results['scoreB']
results['Bwon'] = results['scoreA']<results['scoreB']
results

,date,processor,dayNum,dayOrder,teamA,teamB,scoreA,scoreB,fullTeamA,fullTeamB,Awon,Bwon
0,2019-10-02,Play-In,Day 1,1,CG,UOL,0,1,CLUTCH GAMING,UNICORNS OF LOVE.CIS,False,True
1,2019-10-02,Play-In,Day 1,2,DFM,SPY,0,1,DETONATION FOCUSME,SPLYCE,False,True
2,2019-10-02,Play-In,Day 1,3,MMM,UOL,1,0,MAMMOTH,UNICORNS OF LOVE.CIS,True,False
3,2019-10-02,Play-In,Day 1,4,DFM,ISG,0,1,DETONATION FOCUSME,ISURUS,False,True
4,2019-10-02,Play-In,Day 1,5,MMM,CG,0,1,MAMMOTH,CLUTCH GAMING,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
164,2020-10-11,Main_Event,Day 8,6,DRX,TES,0,1,DRX,TOP ESPORTS,False,True
165,2020-10-15,Main_Event,Quarterfinals,4,DWG,DRX,0,0,DAMWON GAMING,DRX,False,False
166,2020-10-16,Main_Event,Quarterfinals,2,SN,JDG,0,0,SUNING,JD GAMING,False,False
167,2020-10-17,Main_Event,Quarterfinals,1,TES,FNC,0,0,TOP ESPORTS,FNATIC,False,False


In [42]:

############################################
# region/rank
results['regionA'] = ''
results['regionB'] = ''
results['rankA'] = ''
results['rankB'] = ''

for index, row in results.iterrows():
    teamA = row['fullTeamA']
    teamB = row['fullTeamB']
    regionA = rank.loc[rank[rank['teamName']==teamA].index,'region']
    regionB = rank.loc[rank[rank['teamName']==teamB].index,'region']
    
    rankA = rank.loc[rank[rank['teamName']==teamA].index,'rank']
    rankB = rank.loc[rank[rank['teamName']==teamB].index,'rank']
    
    if not regionA.empty:
        results.loc[index, 'regionA'] = regionA.values[0]
        results.loc[index, 'rankA'] = rankA.values[0]
    if not regionB.empty:
        results.loc[index, 'regionB'] = regionB.values[0]
        results.loc[index, 'rankB'] = rankB.values[0]

##################################
# last won
results['Alastwon'] = False
results['Blastwon'] = False
won_last = defaultdict(bool)
for index, row in results.iterrows():
    teamA = row['teamA']
    teamB = row['teamB']
    if index == 0:
        won_last[teamA] = row['Awon']
        won_last[teamB] = row['Bwon']
        continue

    results.loc[index, 'Alastwon'] = won_last[teamA]
    results.loc[index, 'Blastwon'] = won_last[teamB]
    won_last[teamA] = row['Awon']
    won_last[teamB] = row['Bwon']

##################################
# last who won
results['lastAwon'] = False
results['lastBwon'] = False
won_last = defaultdict(bool)
for index, row in results.iterrows():
    teamA = row['teamA']
    teamB = row['teamB']
    if index == 0:
        continue
    for index1, row1 in results[:index].iterrows():
        if row1['teamA'] == teamA and row1['teamB'] == teamB:
            results.loc[index, 'lastAwon'] = results.loc[index1, 'Awon']
            results.loc[index, 'lastBwon'] = results.loc[index1, 'Bwon']
        elif row1['teamA'] == teamB and row1['teamB'] == teamA:
            results.loc[index, 'lastAwon'] = results.loc[index1, 'Bwon']
            results.loc[index, 'lastBwon'] = results.loc[index1, 'Awon']

##################################
# BO
def bo(score):
    max_score = max(score['scoreA'], score['scoreB'])
    if max_score == 1:
        return True
    elif max_score == 3:
        return False
        
results['BO1'] = results[['scoreA','scoreB']].apply(bo, axis=1)

results



,date,processor,dayNum,dayOrder,teamA,teamB,scoreA,scoreB,fullTeamA,fullTeamB,...,Bwon,regionA,regionB,rankA,rankB,Alastwon,Blastwon,lastAwon,lastBwon,BO1
0,2019-10-02,Play-In,Day 1,1,CG,UOL,0,1,CLUTCH GAMING,UNICORNS OF LOVE.CIS,...,True,North America,CIS,3,1,False,False,False,False,True
1,2019-10-02,Play-In,Day 1,2,DFM,SPY,0,1,DETONATION FOCUSME,SPLYCE,...,True,Japan,Europe,1,3,False,False,False,False,True
2,2019-10-02,Play-In,Day 1,3,MMM,UOL,1,0,MAMMOTH,UNICORNS OF LOVE.CIS,...,False,Oceania,CIS,1,1,False,True,False,False,True
3,2019-10-02,Play-In,Day 1,4,DFM,ISG,0,1,DETONATION FOCUSME,ISURUS,...,True,Japan,Latin America,1,1,False,False,False,False,True
4,2019-10-02,Play-In,Day 1,5,MMM,CG,0,1,MAMMOTH,CLUTCH GAMING,...,True,Oceania,North America,1,3,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,2020-10-11,Main_Event,Day 8,6,DRX,TES,0,1,DRX,TOP ESPORTS,...,True,Korea,China,2,1,True,False,False,True,True
165,2020-10-15,Main_Event,Quarterfinals,4,DWG,DRX,0,0,DAMWON GAMING,DRX,...,False,Korea,Korea,1,2,False,False,False,False,None
166,2020-10-16,Main_Event,Quarterfinals,2,SN,JDG,0,0,SUNING,JD GAMING,...,False,China,China,3,2,True,True,False,False,None
167,2020-10-17,Main_Event,Quarterfinals,1,TES,FNC,0,0,TOP ESPORTS,FNATIC,...,False,China,Europe,1,2,True,False,False,False,None


In [43]:
from sklearn.preprocessing import LabelEncoder
encoding = LabelEncoder()
encoding.fit_transform(results['regionA'].values)
regionA = encoding.fit_transform(results['regionA'].values)
regionB = encoding.fit_transform(results['regionB'].values)

region = np.vstack([regionA, regionB]).T

from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()
r = onehot.fit_transform(region).todense()
df = pd.DataFrame(r)

index = results[(results['scoreA']+results['scoreB']>0) & (results['processor']=='Main_Event')].index

#X_previouswins = results.loc[]
df1 = df.loc[index,:]

#X_previouswins = X_previouswins.join(df)
X_previouswins = df1.values


In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
forest = joblib.load('forest.m')


forest = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features=8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=14, verbose=0,
                       warm_start=False)



y_true = results.loc[results[(results['scoreA']+results['scoreB']>0) & (results['processor']=='Main_Event')].index,['Awon']].values
forest.fit(X_previouswins,y_true)
y_pre = forest.predict(X_previouswins)
y_pre

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


array([False,  True,  True,  True, False, False,  True,  True, False,
       False,  True, False, False, False,  True, False,  True,  True,
        True, False,  True,  True, False, False, False, False,  True,
       False,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False,  True,  True,  True, False,
       False, False, False, False,  True,  True, False, False,  True,
        True, False, False,  True,  True,  True,  True, False, False,
       False, False,  True,  True, False,  True, False, False, False,
        True,  True, False, False,  True, False, False, False,  True,
        True, False, False,  True,  True, False,  True])

In [45]:
success = 0
for index in range(y_pre.shape[0]):
    Awon_p = y_pre[index]
    Awon_t = y_true[index]
    if Awon_p == Awon_t:
        success += 1
print('score: {:.2f}%'.format((success/y_pre.shape[0])*100))

score: 73.58%


In [46]:

results_pre = results.loc[results[(results['scoreA']+results['scoreB']==0) & (results['processor']=='Main_Event')].index,['date','dayNum','dayOrder','teamA','teamB', 'Awon', 'Bwon']]
results_pre = results_pre.reset_index()
index = results[(results['scoreA']+results['scoreB']==0) & (results['processor']=='Main_Event')].index
df1 = df.loc[index,:]
X_previouswins = df1.values

y_pre = forest.predict(X_previouswins)

for index in range(y_pre.shape[0]):
    results_pre.loc[index,'Awon'] = y_pre[index]
    results_pre.loc[index,'Bwon'] = not y_pre[index]

results_pre

,index,date,dayNum,dayOrder,teamA,teamB,Awon,Bwon
0,165,2020-10-15,Quarterfinals,4,DWG,DRX,False,True
1,166,2020-10-16,Quarterfinals,2,SN,JDG,True,False
2,167,2020-10-17,Quarterfinals,1,TES,FNC,True,False
3,168,2020-10-18,Quarterfinals,3,GEN,G2,True,False


In [117]:

X_all = np.hstack([X_previouswins, X_teams_expanded])
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_all, y_true, scoring='accuracy')
print('score: {:.2f}%'.format(np.mean(scores)*100))


score: 66.08%


In [126]:
from sklearn.model_selection import GridSearchCV

parameter_space = {
                   "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                   }
clf = DecisionTreeClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_previouswins, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))

Accuracy: 68.2%


In [174]:
forest = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features=8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=14, verbose=0,
                       warm_start=False)
scores = cross_val_score(forest, X_previouswins, y_true, scoring='accuracy')
print('score: {:.2f}%'.format(np.mean(scores)*100))
from sklearn.externals import joblib
joblib.dump(forest, 'forest.m')
joblib.load('forest.m')

score: 72.94%


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features=8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=14, verbose=0,
                       warm_start=False)